# Download MODIS with pyModis & create NDVI

## Install libraries

In [ ]:
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y


In [ ]:
!pip install rasterio
!pip install pyModis
!pip install zkyhaxpy
!pip install geopandas


## Import libraries

In [ ]:

import time
import seaborn as sns
import pandas as pd
import numpy as np
from glob import glob
import re
import random
import datetime
from tqdm.notebook import tqdm
import os, shutil
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup

from zkyhaxpy import io_tools, console_tools, colab_tools, gis_tools
import geopandas as gpd
import rasterio
from rasterio import plot
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.mask import mask
import gdal
import pytz

%matplotlib inline




In [ ]:
colab_tools.mount_drive()
colab_tools.authen_gcp()

## Define paths

In [ ]:
folder_modis_raw = '/temp/modis/raw'
folder_modis_ndvi = '/temp/modis/ndvi'
folder_modis_ndvi_reproj = '/temp/modis/ndvi_reproj'
folder_modis_reference = '/temp/modis/reference'
io_tools.create_folders(folder_modis_raw, folder_modis_ndvi, folder_modis_ndvi, folder_modis_reference)


!gsutil cp -r gs://unbdh2022-multiverseofdata-dev/modis/reference /temp/modis


In [ ]:
!wget https://data.humdata.org/dataset/d24bdc45-eb4c-4e3d-8b16-44db02667c27/resource/d0c722ff-6939-4423-ac0d-6501830b1759/download/tha_adm_rtsd_itos_20210121_shp.zip -O /temp/tha_adm_rtsd_itos_20210121_shp.zip

!unzip /temp/tha_adm_rtsd_itos_20210121_shp.zip -d /temp/shapefile


In [ ]:
thailand_shp_path = os.path.join('/temp/shapefile/tha_admbnda_adm0_rtsd_20220121.shp')

## Download raw image of MOD13Q1

In [ ]:
#Username & Password registered from https://lpdaac.usgs.gov/products/mod13q1v006/
user = input('User')
password = input('Password')

In [ ]:
#Select MODIS' scenes that cover Thailand
download_script = f'modis_download.py -U {user} -P {password} -r -t h27v06,h27v07,h27v08,h28v07,h28v08 -p MOD13Q1.006 -f 2000-01-01 -e 2022-12-31 "{folder_modis_raw}"'
console_tools.execute_cmd(download_script)



## MOSAIC into one raster covering Thailand

In [ ]:
gdf_thailand = gpd.read_file(thailand_shp_path)
th_shape = gdf_thailand.geometry.iloc[0]

In [ ]:
def mosaic_and_masked_thailand(tmp_folder, idx_nm, out_path):
    dst_crs = {'init': 'EPSG:4326'}
    mosaic_4326_path = os.path.join(tmp_folder, f'mosaic-{idx_nm}-4326.tif')

    list_files = io_tools.get_list_files_re(tmp_folder, idx_nm)
    str_list_files = ' '.join(list_files)
    mosaic_path = os.path.join(tmp_folder, f'mosaic-{idx_nm}-ori.tif')
    gdal_merge_cmd = f'gdal_merge.py -o {mosaic_path} {str_list_files}'
    result_code = os.system(gdal_merge_cmd)
    assert(result_code==0)


    #reproject mosaic to 4326
    with rasterio.open(mosaic_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(mosaic_4326_path, mode='w', **kwargs) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

    #masked with thailand boundary
    with rasterio.open(mosaic_4326_path) as src:
        out_image, transform = mask(src, th_shape, crop=True, pad=True)
        out_meta = src.meta.copy()
        height, width = out_image[0].shape
        out_meta.update({'transform' : transform,
                          'width': width,
                          'height': height})

    with rasterio.open(out_path, "w", **out_meta) as dst:
        dst.write(out_image)    

    print(f'{out_path} has been created.')


# Calculate NDVI and mosaic all scenes covering Thailand

In [ ]:
list_raw_path = io_tools.get_list_files_re(folder_modis_raw, '.hdf$')

df_list_raw_path = pd.DataFrame(list_raw_path, columns=['filepath'])
df_list_raw_path['filename'] = df_list_raw_path['filepath'].str.split('/', expand=True).iloc[:, -1]
df_list_raw_path['scene_id'] = df_list_raw_path['filename'].str.split('.', expand=True).iloc[:, 2]
df_list_raw_path['year'] = df_list_raw_path['filename'].str.split('.', expand=True).iloc[:, 1].str.slice(1, 5).astype(int)
df_list_raw_path['day_of_year'] = df_list_raw_path['filename'].str.split('.', expand=True).iloc[:, 1].str.slice(5, 8).astype(int)
df_list_raw_path['cnt'] = 1

df_list_raw_path = pd.DataFrame(list_raw_path, columns=['filepath'])
df_list_raw_path['filename'] = df_list_raw_path['filepath'].apply(lambda filepath : os.path.basename(filepath))
df_list_raw_path[['product_id', 'date', 'scene_id', 'product_version', 'rev_date', 'fileextension']] = df_list_raw_path['filename'].str.split('.', expand=True)
df_list_raw_path['date'] = df_list_raw_path['date'].str.slice(1,8).astype(int)

list_error_log = []
for date, df_list_raw_path_curr_date in tqdm(df_list_raw_path.groupby('date', sort=False), 'Iterate : date'):     
    tmp_folder = os.path.join('/tmp/', f'{date}')
    os.makedirs(tmp_folder, exist_ok=True)
    error_f = False
    year = int(np.floor(date / 1000))
    days = date % 1000
    date_reformat = (datetime.datetime(year, 1, 1) + datetime.timedelta(days - 1))
    date_yyyymmdd = datetime.datetime.strftime(date_reformat, '%Y%m%d')

    path_modis_ndvi = os.path.join(folder_modis_ndvi, str(year), f'mod250m16d-ndvi-{date_yyyymmdd}.tif')
    io_tools.create_folders(path_modis_ndvi)
  
    if (os.path.exists(path_modis_ndvi)==True):
        print(f'{path_modis_ndvi} already exists. Skip these files')        
        print()
        continue

    
    print(f'Processing NDVI {date}...')

    #calculate NDVI for each scene
    nbr_scenes = 0
    for tuples_row in df_list_raw_path_curr_date.itertuples():
        
        raw_path = tuples_row.filepath
        scene_id = tuples_row.scene_id        
        print(f'Processing Scene : {scene_id}')

        srcfile = f'HDF4_EOS:EOS_GRID:"{raw_path}":MODIS_Grid_16DAY_250m_500m_VI:250m 16 days NDVI' 
        destfile = os.path.join(tmp_folder, f'{scene_id}-ndvi.tif')
        gdal_cmd = "gdal_translate -of GTiff '" + srcfile + "' " +  destfile        
        gdal_result = os.system(gdal_cmd)
        if gdal_result !=0:
            error_f = True
            print(f'{gdal_cmd} failed!')
            list_error_log.append(f'{gdal_cmd} failed!')
            break
        nbr_scenes += 1

    #check if error found
    if error_f == True:
        print(f'Found error in {date}')        
        continue
    elif nbr_scenes != 5:
        print(f'Found missing scenes in {date}')        
        list_error_log.append(f'Found missing scenes in {date}')
        continue
        
    #mosaic and masked Thailand boundary   
    #create raster for NDVI       
    if os.path.exists(path_modis_ndvi)==True:
        print(f'{path_modis_ndvi} already exists. Skip this file')        
    else:
        mosaic_and_masked_thailand(tmp_folder, 'ndvi', path_modis_ndvi) 

    #delete temp files
    list_tmp_files = io_tools.get_list_files_re(tmp_folder)
    for filepath in list_tmp_files:
        os.remove(filepath)
    os.removedirs(tmp_folder)

    print(f'Finished NDVI for {date}.')
    print()     
    print(f'Current error found : {len(list_error_log)} files')
    print()
    #assert(False)


print()
if len(list_error_log) > 0:
    for error_msg in list_error_log:
        print(error_msg)

## Reproject MODIS NDVI raster into EPSG:4326

In [ ]:
#Get a list of all mosaiced' MODIS NDVI rasters 
df_list_files_modis = io_tools.get_list_files_re(folder_modis_ndvi, '.tif$', return_df=True)
df_list_files_modis['img_date_int'] = df_list_files_modis['file_nm'].str.findall('\d{8}').apply(lambda val: int(val[0]))
df_list_files_modis['img_date'] = df_list_files_modis['file_nm'].str.findall('\d{8}').apply(lambda val: f'{val[0][:4]}-{val[0][4:6]}-{val[0][6:]}')
df_list_files_modis['img_year'] = df_list_files_modis['img_date'].str.slice(0, 4).astype(int)

path_modis_reproj_ref = os.path.join(folder_modis_reference, 'mod250m16d-ndvi-reproj.tif')

for _, s_path_modis_info in df_list_files_modis.iterrows():
    path_modis_src = s_path_modis_info.file_path
    file_name = os.path.basename(path_modis_src).replace('no-reproj', 'reproj')
    year = s_path_modis_info.img_year
    path_modis_dest = os.path.join(folder_modis_ndvi_reproj, str(year), file_name)
    io_tools.create_folders(path_modis_dest)
    if '(' in file_name:
        print(f'Skip {file_name}')
        continue
    elif os.path.exists(path_modis_dest):
        print(f'{path_modis_dest} already exists. Skip')
        continue
    
    #Reproject MODIS Sinusoidal into EPSG:4326 using a reference GeoTiff file with EPSG:4326 of 200m x 200m resolution
    gis_tools.reproject_raster_from_ref(path_modis_src, path_modis_dest, path_modis_reproj_ref)

# Save in GCS bucket
!gsutil -m cp -r -n /temp/modis/ndvi_reproj gs://unbdh2022-multiverseofdata-dev/modis    